# P04: Introdução ao Spark com Python

## Operações de conjuntos em RDDs: `subtract`, `distinct` e `cartesian`.

O Spark suporta muitas das operações que temos em conjuntos matemáticos, como união e intersecção. 

Essas operações exigem que os RDDs que estão sendo operados sejam do mesmo tipo.

## Como obter o `SparkContext`

O `SparkContext` Carregado automaticamente quando o notebook é iniciado pelo PySpark

In [ ]:
# Somente necessário quando o notebook não é carregado pelo PySpark
#import pyspark
#sc = pyspark.SparkContext(appName="P4")

# Quando o notebook é carregado pelo PySpark, a variável sc é disponibilizada automaticamente
sc

## Obtendo o conjunto de dados de análise reduzido e criando o RDD

Usaremos um conjunto reduzido de dados (10%) da Copa KDD de 1999, que contém quase meio milhão de registros. O arquivo é fornecido como um *Gzip*.

In [ ]:
import urllib.request as request
f = request.urlretrieve("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

Atenção! Lembre-se de colocar o arquivo baixado no `HDFS` (caso ainda não tenha feito). Além disso, inicie o `HDFS` e o `Yarn`.

```bash
start-dfs.sh
start-yarn.sh
hdfs dfs -put kddcup.data_10_percent.gz /usr/hduser
```

In [ ]:
nome_arquivo = "./kddcup.data_10_percent.gz"
rdd = sc.textFile(nome_arquivo)

## Obtendo as interações de ataque utilizando `subtract`

Suponha que já temos as interações de rede que não configuram ataque (normal) de uma análise prévia.

In [ ]:
normal_rdd = rdd.filter(lambda x: "normal." in x)

As interações de ataque poderiam ser obtidas subtraindo-se as interações normais do RDD sem filtragem.

In [ ]:
ataque_rdd = rdd.subtract(normal_rdd)

Vamos fazer alguma aritmética para checar os resultados

In [ ]:
from time import time

# Contando todas as interações
t0 = time()
contagem_total = rdd.count()
tt = time() - t0
print("A contagem demorou {} segundos".format(round(tt,3)))

In [ ]:
# Contando as interações normais
t0 = time()
contagem_normal = normal_rdd.count()
tt = time() - t0
print("Contagem dos normais em {} segundos".format(round(tt,3)))

In [ ]:
# Contando as interações de ataque
t0 = time()
contagem_ataque = ataque_rdd.count()
tt = time() - t0
print("Contagem dos ataques em {} segundos".format(round(tt,3)))

In [ ]:
print("Existem {} interações normais e {} interações de ataque, \
de um total de {} interações".format(contagem_normal, contagem_ataque, contagem_total))

## A transformação `cartesian`

A transformação `cartesian` retorna todos os pares possíveis de elementos entre dois RDDs. No nosso caso, retornará todas as combinações possíveis entre serviços e protocolos utilizados nas interações de rede.


Vamos isolar cada coleção de valores em dois RDDs separados. Para isso, vamos usar o `distinct` no conjunto de dados CSV. Da [descrição do conjunto de dados](http://kdd.ics.uci.edu/databases/kddcup99/kddcup.names), sabamos que o protocolo é a segunda coluna e o serviço a terceira.

Vamos obter os protocolos.

In [ ]:
dados_csv = rdd.map(lambda x: x.split(","))
protocolos_rdd = dados_csv.map(lambda x: x[1]).distinct()
protocolos_rdd.collect()

Vamos fazer o mesmo com os serviços.

In [ ]:
servicos_rdd = dados_csv.map(lambda x: x[2]).distinct()
servicos_rdd.collect()

Agora podemos fazer o produto cartesiano

In [ ]:
combinacoes = protocolos_rdd.cartesian(servicos_rdd).collect()
print("Existem {} combinacoes de protocolo X serviço".format(len(combinacoes)))